In [1]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

print(current_datetime)

2024-11-19 14:20:14.221979


# Spatio Temporal Logistic
This notebook is the first test of a spatio temporal logistic to link properly the **built-up surface** of a given region depending on its quality of life (**GDP/cap**) and of its **population density**.

In [2]:
import numpy as np
from Region import Region
from Df import Df
import pandas as pd
import geopandas as gpd
import os

In [3]:
from concurrent.futures import ThreadPoolExecutor
import itertools
from colorama import Fore, Style# init

In [4]:
max_workers = 8

## Part 0 : Theory
First we define some classical functions that will be used later for our modelisation.

### The logistic function
$f(t) = \frac{S}{1+e^{(-k(t-t_0))}}$

with: 
- *S* : the saturation level
- *k* : the slope
- $t_0$ : the half height value (left right alignement).

The logistic function serves as a predictive equation and as the motor of our dynamical models. Indeed, we observe that at the country scale, as time passes, the GDP/cap increases and some observables as the number of cars or as the number of m2/cap. This means that the quality of life, and thus the stocks consumed by a region improve. But this exponential growth does not increase indefinetely. A saturation is observed even if the GDP/cap still increase. This is explained because people usually do not own 2 or 3 cars even though they possess the money to. An logistic function, also known as the S curve, is a way to model this observation.

### The exponential decay function
$f(x) = a\, e^{-b\,x}+c$

with:
- *a* : the slope
- *b* : the half-life
- *c* : the bias.

The exponential decay is a decreasing exponential observed in nature (for example for the probability of nuclear decay over time). This is one of our assumption to model the decreasing phenomena observed betwenn the **built-up surface/cap** and the **population density** of a region.

In [5]:
def logistic(x, a, b, c):
    return a / (1 + np.exp(-b*(x-c)))

def exponential_decay(X, a, b, c):
    return a * np.exp(-b * X) + c

def exponential_decay2(X, a, b):
    return a * np.exp(-b * X) 

### The Spatio Temporal Logistic function
$f(t,x) = \frac{a\,e^{-b\,x}+c}{1 + e^{(-k(t-t_0))}}$

The Spatio Temporal Logistic function (**STL**) is the mix between our classical logistic expression and the exponential decay as S (the saturation level).

In [6]:
def STL(X, a, b, c, d, e):
    # STL stands for Spatio Temporal Logistic. It is an refinment of a logistic, dependant on time (here the GDP per capita serves 
    # as proxy for time), with a saturation level which is a function of space (here the population density of a region)
    x1, x2 = X
    saturation = exponential_decay(x2, a, b,c)
    return saturation / (1 + np.exp(-d*(x1-e)))

## PART 1 : Initialisation
Initialisation of the analysis parameters.

- **region_names** : (*string*) the country to study, named by their ISO3 
- **years** : (*string*) years to study
- **raster_S** : (*string*) letter used in the **GHSL** dataset (S, S_NRES, POP, ...)
- **lvl** : (*int*) the level of our administrative data (GDP and population)
- **subregion_borders** : (*string*) the path to administrative border shapefile to cut the subregions 
- **i dentifier** : (*string*) the column name to match the region names between the administrative data and the GIS data

To each region is associated a **DataFrame** (*oecd_DF_merged*) with the subregions matching administrative observables (GDP, population, etc).

In [7]:
lvl = 1

# GHSL type
raster_str = "Built_S"
with_parents_computation = False

In [8]:
data_folder = "/data/mineralogie/hautervo/data/"
folder_GHSL_S = data_folder + "Outputs/GHSL/Built_S/TL" + str(lvl) + "/"
folder_GHSL_POP = data_folder + "Outputs/GHSL/POP/TL" + str(lvl) + "/"
folder_OSM_building = data_folder + "Outputs/OSM/building/TL" + str(lvl) + "/"

In [10]:
# # Make the OECD complete DF
# oecd_gdp_per_cap_file = data_folder + r"OECD/GDP per capita/TL" + str(lvl) + r"/Gross Domestic Product per capita, in USD.csv"
# oecd_population_file = data_folder + r"OECD/Population/TL" + str(lvl) + r"/Resident population.csv"

# oecd_gdp_per_capita_df = pd.read_csv(oecd_gdp_per_cap_file, skiprows=0, header=1)
# oecd_population_df = pd.read_csv(oecd_population_file, skiprows=0, header=1)

# df_list = []
# df_list.append(oecd_gdp_per_capita_df)
# df_list.append(oecd_population_df)

# subregion_col = "tl"+str(lvl)+"_id"
# parent_col = "iso3"

# for df in df_list:
#     df[subregion_col] = df["Code"]

In [11]:
# The OECD admin units

oecd_admin_units = data_folder + "OECD/admin_units/TL" + str(lvl) + "/OECD_TL" + str(lvl) + "_2020_ESRI54009.shp"
gpd_oecd_admin_units = gpd.read_file(oecd_admin_units)

DriverError: Failed to open dataset (flags=68): /data/mineralogie/hautervo/data/OECD/admin_units/TL1/OECD_TL1_2020_ESRI54009.shp

In [12]:
# Countries to ignore from our study (not enough data)
country_to_pop = ["SRB", "CRI", "ISR", "CYP", "ISL", "ALB", "LIE","MNE","MKD"]

In [ ]:
# regions_names = list(oecd_gdp_per_capita_df["Country"].unique())

# #exclude some countries if necessary
# for c in country_to_pop:
#     regions_names.pop(regions_names.index(c))

regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "LUX", "ESP", "USA", "JPN", "CAN", "AUS"] # to remove
# regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "LUX", "ESP"] # to remove
# regions_names = ["FRA"]

years = ["1975", "1990", "2000", "2010", "2020"] 
# years = ["2000", "2010", "2020"]
# years = ["2020"]

regions = []

for name in regions_names:
    new_region = Region(name, lvl-1)
    new_region.parent_name = name
    new_region.subregions.append(new_region)
    regions.append(new_region)

    for y in years:        
        new_region.add_gis(data_folder + "GHSL/"+ raster_str + "/E" + y + "_100m_Global/subregions/" + name + ".tif", raster_str + "_" + y, str(y), lvl-1) 
        new_region.add_gis(data_folder + "GHSL/Built_POP/E" + y + "_100m_Global/subregions/" + name + ".tif", "POP_" + y, str(y), lvl-1) 



## PART 2 : Computation of the observables
Now that we define all the parameters of our study, we will cut the regions into their respective subregions (*make_subregions*). 

We then compute for each subregions GIS, the geographical observables that we store in its oecd_DF_merged.

In [30]:
def get_GHSL_values(subregion, folder: str, type: str, overwrite):    
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name, '_'.join(years))+".csv"
    if not os.path.isfile(csv_path) or overwrite:
        # S
        if type == "S":
            output_df = pd.DataFrame({"year": years, "Built up surface GHSL":None, "Total surface":None, "Built up surface fraction":None})

            for y in years:
                # first use the Built_S gis
                gis = next((gis for gis in subregion.gis_list if gis.name == raster_str + "_" + y), None)
                if gis != None:
                    output_df.loc[output_df["year"]==y, "Built up surface GHSL"] = int(gis.get_total_sum_pixel_values())
                    output_df.loc[output_df["year"]==y, "Total surface"] = int(1e4*gis.get_total_number_pixels())
                    output_df.loc[output_df["year"]==y, "Built up surface fraction"] = output_df.loc[output_df["year"]==y, "Built up surface GHSL"] / output_df.loc[output_df["year"]==y, "Total surface"]
                else:
                    print(Fore.RED, f"{subregion.name} ", raster_str + "_" + y, " not found.", Style.RESET_ALL)

            # save the new df
            os.makedirs(os.path.dirname(csv_path), exist_ok="True")
            output_df.to_csv(csv_path, index=False)
            subregion.output_df_list.append(Df(output_df, type))

        # POP    
        elif type == "POP":
            output_df = pd.DataFrame({"year": years, "Population":None})

            for y in years:
                # first use the Built_S gis
                gis = next((gis for gis in subregion.gis_list if gis.name == "POP_" + y), None)
                if gis != None:
                    output_df.loc[output_df["year"]==y, "Population"] = int(gis.get_total_sum_pixel_values())
                else:
                    print(Fore.RED, f"{subregion.name} ", "POP_" + y, " not found.", Style.RESET_ALL)

            # save the new df
            os.makedirs(os.path.dirname(csv_path), exist_ok="True")
            output_df.to_csv(csv_path, index=False)
            subregion.output_df_list.append(Df(output_df, type))

        # If we fall here, something wrong happened    
        else:
            print(f"Type of GHSL data to compute : {type} not understood.")
    else:
        #use a precomputed csv
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), type))

In [31]:
def get_OSM_areas(subregion, folder: str, overwrite=False): 
    gis_name = "OSM_building"  
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name,  subregion.name +".csv")

    if not os.path.isfile(csv_path) or overwrite:
        gis = next((gis for gis in subregion.gis_list if gis.name == gis_name), None)
        if gis is not None:
            try:
                shp = gpd.read_file(gis.file)
                print("Starting area OSM ", gis.file)
                value = (shp["geometry"].area).sum()
                output_df = pd.DataFrame({gis_name+"_area":value}, index=[0])

                # save the new df
                os.makedirs(os.path.dirname(csv_path), exist_ok="True")
                output_df.to_csv(csv_path, index=False)
                subregion.output_df_list.append(Df(output_df, gis_name))
                print("Ending area OSM ", gis.file)
            except Exception as e:
                print(e)
        else:
            print(f"OSM shp for {subregion.name} not found.")
    else:
        #use a precomputed csv
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), gis_name))
    

### Preprocess with the desired computational methods

In [ ]:
def preprocess():
    # Step 1 : make subregions
    overwrite = False

    # print(Fore.GREEN + "Starting make_subregions()" + Style.RESET_ALL)
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     list(executor.map(lambda region: region.make_subregions(gpd_oecd_admin_units, subregion_col, parent_col, overwrite=overwrite), regions))

    # Step 2.1 : Computation
    overwrite = False

    subregions_list_parallel = regions

    # for region in regions:
    #     for subregion in region.subregions:
    #         subregions_list_parallel.append(subregion)
    #         try:
    #             file = os.path.join(folder_OSM_building, subregion.parent_name, subregion.name, subregion.name +".shp")
    #             if os.path.isfile(file):
    #                 subregion.add_gis(file, "OSM_building", "", "")
    #             else:
    #                 print(f"File {file} not found.")
    #         except Exception as e:
    #             print(e)

    print(Fore.GREEN + "Starting GHSL_S" + Style.RESET_ALL)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(get_GHSL_values, subregions_list_parallel, itertools.repeat(folder_GHSL_S), itertools.repeat("S"), itertools.repeat(overwrite))
    print(Fore.GREEN + "Starting GHSL_POP" + Style.RESET_ALL)   
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(get_GHSL_values, subregions_list_parallel, itertools.repeat(folder_GHSL_POP), itertools.repeat("POP"), itertools.repeat(overwrite))

    # print(Fore.GREEN + "Starting OSM_area_computation()" + Style.RESET_ALL)
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     executor.map(get_OSM_areas, subregions_list_parallel, itertools.repeat(folder_OSM_building), itertools.repeat(overwrite))

    del subregions_list_parallel # not needed anymore

    if with_parents_computation:
        print(Fore.GREEN + "Starting computing region DF" + Style.RESET_ALL)
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            executor.map(lambda region: region.compute_own_df(years, "GHSL_OECD"), regions)

In [33]:
preprocess()   

Starting GHSL_S
Starting GHSL_POP


In [34]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()
print(Fore.GREEN, "Ended normally.", Style.RESET_ALL)
print(current_datetime)

 Ended normally. 
2024-11-19 14:59:44.744321


# TEST ZONE

In [35]:
# for region in regions:
#     for subregion in region.subregions:
#         print(len(subregion.output_df_list))